In [72]:
import nltk
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [73]:
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import json
import pickle

import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import SGD
import random

In [67]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [74]:
words=[]
classes = []
documents = []
ignore_words = ['?', '!']
data_file = open('/content/drive/MyDrive/intents.json').read()
intents = json.loads(data_file)


In [75]:
for intent in intents['intents']:
    for pattern in intent['patterns']:
        w = nltk.word_tokenize(pattern)
        words.extend(w)
        # adding documents
        documents.append((w, intent['tag']))

        # adding classes to our class list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])


In [76]:
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))

classes = sorted(list(set(classes)))

print (len(documents), "documents")

print (len(classes), "classes", classes)

print (len(words), "unique lemmatized words", words)

910 documents
255 classes ['     Support Vector Machine', '   Decision Trees', '  Perceptron', '  Regularization in Machine Learning', '  applications of supervised learning', '  basic data structures', " 'action' in reinforcement learning", " 'agent' in reinforcement learning", " 'value' in reinforcement learning", ' Classification model', ' Gradient Descent', ' Inductive Logic Programming', ' Polynomial Regression', ' Regularization', ' Ridge Regression technique', ' Types of machine learning', ' advantages of using Polynomial Regression over Linear Regression', ' applications of self-supervised learning', ' components of Bayesian logic program', ' difference between heuristic for rule learning and heuristics', ' disadvantages of supervised learning', ' imbalanced datasheet', ' issues to consider in supervised learning', ' standard supervised learning', 'A/B Testing', 'Algorithm techniques in Machine Learning', 'Appreciation', 'Apriori algorithm', 'Areas of Problems', 'BiasML', 'Bina

In [77]:
pickle.dump(words,open('words.pkl','wb'))
pickle.dump(classes,open('classes.pkl','wb'))

In [78]:
training = []
output_empty = [0] * len(classes)
for doc in documents:
    # initializing bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # lemmatize each word - create base word, in attempt to represent related words
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    # create our bag of words array with 1, if word match found in current pattern
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)

    # output is a '0' for each tag and '1' for current tag (for each pattern)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1

    training.append([bag, output_row])
# shuffle our features and turn into np.array
random.shuffle(training)
training = np.array(training, dtype=object)
# create train and test lists. X - patterns, Y - intents
train_x = [item[0] for item in training]
train_y = [item[1] for item in training]
print("Training data created")

Training data created


In [82]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import SGD
# Assuming you have your training data in train_x and train_y
# train_x: list of input features
# train_y: list of corresponding target labels

# Create the model
model = Sequential()
if train_x:  # Check if train_x is not empty
        model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
else:
        print("Error: train_x is empty. Check data loading and preprocessing.")
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
if train_y:  # Check if train_y is not empty
        model.add(Dense(len(train_y[0]), activation='softmax'))
else:
        print("Error: train_y is empty. Check data loading and preprocessing.")

# Compile the model
sgd = SGD(learning_rate=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

# Train the model
history = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)

# Evaluate the model
loss, accuracy = model.evaluate(np.array(train_x), np.array(train_y), verbose=0)
print(f"Accuracy: {accuracy * 100:.2f}%")

Epoch 1/200
182/182 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.0031 - loss: 5.5485
Epoch 2/200
182/182 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.0273 - loss: 5.5027
Epoch 3/200
182/182 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.0226 - loss: 5.4419
Epoch 4/200
182/182 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.0294 - loss: 5.3266
Epoch 5/200
182/182 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.0400 - loss: 5.1340
Epoch 6/200
182/182 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.0843 - loss: 4.8225
Epoch 7/200
182/182 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.0796 - loss: 4.5448
Epoch 8/200
182/182 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.1267 - loss: 4.2223
Epoch 9/200
182/182 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.1271 - loss: 3.9597
Epoch 10/200
182/182 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.1661 - loss: 3.7420
Epoch 11/200
182/182 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.1686 - loss: 3.5548
Epoch 12/200
182/182 ━━━━━━━━━━━━━━━━━━━━